![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

##  Description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> est un moteur de recherche de voyages qui aide les utilisateurs à planifier leur prochain voyage au meilleur prix.

L'entreprise a été fondée en 2004 par Steve Hafner et Paul M. English. Après quelques tours de financement, Kayak a été acquis par Booking Holdings, qui détient désormais

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

Avec un chiffre d'affaires annuel de plus de 300 millions de dollars, Kayak opère dans presque tous les pays et dans toutes les langues pour aider ses utilisateurs à réserver des voyages à travers le monde.

## Projet 🚧

L'équipe marketing a besoin d'aide sur un nouveau projet. Après avoir effectué des recherches utilisateur, l'équipe a découvert que **70% de leurs utilisateurs qui planifient un voyage aimeraient avoir plus d'informations sur la destination vers laquelle ils se dirigent.**. 

De plus, les recherches utilisateur montrent que **les gens ont tendance à être méfiants à propos des informations qu'ils lisent s'ils ne connaissent pas la marque** qui a produit le contenu.. 

Par conséquent, l'équipe marketing de Kayak souhaite créer une application qui recommandera aux gens où ils devraient planifier leurs prochaines vacances. L'application devrait être basée sur des données réelles concernant :

* La météo
* Les hôtels dans la région

L'application devrait ensuite être en mesure de recommander les meilleures destinations et hôtels

## Goals 🎯

* Scraping des données des destinations 
* Obtention des Données météorologiques
* Récupération des Infos sur les Hôtels
* Stockage des données dans un Data Lake
* Extraction, Transformation et Chargement (ETL) des données

## Scope of this project 🖼️

L'équipe marketing souhaite se concentrer d'abord sur les meilleures villes où voyager en France.  <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> Voici les 35 meilleures villes à visiter en France.: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Votre équipe devrait se concentrer uniquement sur les villes mentionnées ci-dessus pour votre projet. 

# Import des librairies

In [ ]:
!pip install folium


In [1]:
import requests
import pandas as pd
import plotly.express as px
import boto3
import ast
import json
import folium

from io import BytesIO
from branca.element import Figure
from sqlalchemy import create_engine, text

# Obtention des données en utilisant les API

In [2]:
# Liste des villes pour lesquelles nous voulons obtenir des coordonnées GPS
cities = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris",
    "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar",
    "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
    "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille",
    "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes",
    "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege",
    "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"
]

### 1. Obtention des coodonnées GPS des villes via l'API Nominatim. 

Dans cette étape, Nous utilisons l'API Nominatim d'OpenStreetMap pour obtenir les coordonnées géographiques (latitude et longitude) de diverses villes. Pour cela, les espaces dans les noms des villes seront remplacés par des '+' afin d'améliorer la lisibilité des URL de requête. 

Chaque ville sera requêtée via une méthode GET, et les réponses obtenues, contenant les coordonnées, seront converties en JSON. 

Les données pertinentes seront ensuite extraites et stockées dans un DataFrame pandas. Ce processus sera répété pour chaque ville, compilant toutes les coordonnées dans le DataFrame coordinates_df.


In [3]:
# Fonction pour interroger l'API Nominatim et obtenir les coordonnées GPS
def get_coordinates(city):
    # Remplace les espaces par des caractères '+'
    city = city.replace(' ', '+')
    # Défini l'URL de requête avec la ville spécifiée
    url = f"https://nominatim.openstreetmap.org/search?city={city}&format=json"
    
    # Exécute la requête GET à l'API Nominatim
    response = requests.get(url)
    
    # Vérifie si la requête a réussi
    if response.status_code == 200:
        # Obtenir les données JSON de la réponse
        data = response.json()
        return data[0]['lat'], data[0]['lon']
    else:
        return None, None

In [4]:
# Création d'un DataFrame pour stocker les informations des coordonnées GPS
coordinates_df = pd.DataFrame(columns=['City', 'Latitude', 'Longitude'])

In [5]:
# Boucle sur la liste des villes pour obtenir les coordonnées pour chaque ville

for city in cities:
    lat, lon = get_coordinates(city)
    new_row = pd.DataFrame({
        'City': [city],
        'Latitude': [lat],
        'Longitude': [lon]
    })
    coordinates_df = pd.concat([coordinates_df, new_row], ignore_index=True)

coordinates_df

,City,Latitude,Longitude
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,49.314695,-96.9538228
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658
5,Paris,48.8588897,2.3200410217200766
6,Amiens,49.8941708,2.2956951
7,Lille,50.6365654,3.0635282
8,Strasbourg,48.584614,7.7507127
9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923


### 2. Obtention de données de temps (weather) par le biais de l'api open-weather.

Nous utilisons l'API One Call d'OpenWeatherMap pour récupérer les données météorologiques complètes (latitude et longitude) en utilisant une clé API obtenue après inscription sur leur site. 

Après avoir construit l'URL de requête avec les coordonnées géographiques spécifiques, nous effectuons une requête GET pour chaque ensemble de coordonnées. 

Les réponses obtenues seront converties en JSON, et les données météorologiques pertinentes seront extraites et stockées dans une colonne 'Weather' du DataFrame coordinates_df. 

Ce processus sera répété pour chaque ligne du DataFrame, et enfin, les données seront sauvegardées dans un fichier CSV coordinates.csv.

In [6]:
# Clé API obtenu suite à inscription sur le site
api_key = # Votre clé API Open-weather

In [7]:
# Fonction pour récupérer les données météorologiques en utilisant l'API OpenWeatherMap. 
def get_weather(lat, lon):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=minutely,hourly,alert&units=metric&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        weather_data = response.json()
        return weather_data
    else:
        print(f"Failed to get weather data: Status code {response.status_code}, Response: {response.text}")
        return None


In [8]:
# Applique la fonction à chaque ligne de `coordinates_df` pour récupérer les données météorologiques détaillées.
coordinates_df['Weather'] = coordinates_df.apply(lambda row: get_weather(row['Latitude'], row['Longitude']), axis=1)

coordinates_df

,City,Latitude,Longitude,Weather
0,Mont Saint Michel,48.6359541,-1.511459954959514,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E..."
1,St Malo,49.314695,-96.9538228,"{'lat': 49.3147, 'lon': -96.9538, 'timezone': ..."
2,Bayeux,49.2764624,-0.7024738,"{'lat': 49.2765, 'lon': -0.7025, 'timezone': '..."
3,Le Havre,49.4938975,0.1079732,"{'lat': 49.4939, 'lon': 0.108, 'timezone': 'Eu..."
4,Rouen,49.4404591,1.0939658,"{'lat': 49.4405, 'lon': 1.094, 'timezone': 'Eu..."
5,Paris,48.8588897,2.3200410217200766,"{'lat': 48.8589, 'lon': 2.32, 'timezone': 'Eur..."
6,Amiens,49.8941708,2.2956951,"{'lat': 49.8942, 'lon': 2.2957, 'timezone': 'E..."
7,Lille,50.6365654,3.0635282,"{'lat': 50.6366, 'lon': 3.0635, 'timezone': 'E..."
8,Strasbourg,48.584614,7.7507127,"{'lat': 48.5846, 'lon': 7.7507, 'timezone': 'E..."
9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,"{'lat': 48.2495, 'lon': 7.3455, 'timezone': 'E..."


In [9]:
# Transformation de données en fichier csv
coordinates_df.to_csv('coordinates.csv', index=False)

### 3. Déterminer les 7 villes ayant le temps le plus beau.

Nous utilisons les données météorologiques collectées précédemment pour calculer les scores de température moyenne et de précipitation totale de chaque ville. La fonction calculate_weather_scores extrait et somme les températures diurnes et les précipitations (si présentes) pour chaque jour des données météo fournies. Ces scores sont ensuite ajoutés au DataFrame coordinates_df.

Après avoir calculé ces scores pour chaque ville, nous trions les villes pour identifier celles avec les températures les plus élevées et les précipitations les plus faibles, favorisant ainsi les destinations avec un climat agréable. 

Les cinq premières villes, qualifiées de destinations ayant le meilleur temps, seront ensuite visualisées sur une carte centrée sur la France. 

Cette carte, générée avec Plotly, mettra en évidence ces villes en fonction de leur température moyenne et de leurs précipitations, offrant ainsi une représentation graphique  des meilleures destinations selon le climat.

In [10]:
# Fonction de calcule de la température moyenne et la précipitation totale sur la période couverte par les données météorologiques.
def calculate_weather_scores(weather_data):
    total_temp = 0
    total_precipitation = 0
    for day in weather_data['daily']:
        total_temp += day['temp']['day']
        # Ajoute les précipitations seulement si elles sont prévues
        total_precipitation += day.get('rain', 0)
    average_temp = total_temp / len(weather_data['daily'])
    return average_temp, total_precipitation

In [11]:
# Applique la fonction pour chaque ligne de `coordinates_df et stock les résultats dans les colonnes 'AverageTemp' et 'TotalPrecipitation'.
coordinates_df[['AverageTemp', 'TotalPrecipitation']] = coordinates_df.apply(lambda row: calculate_weather_scores(row['Weather']), axis=1, result_type='expand')

In [12]:
coordinates_df.head()

,City,Latitude,Longitude,Weather,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,48.6359541,-1.511459954959514,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",18.28375,23.87
1,St Malo,49.314695,-96.9538228,"{'lat': 49.3147, 'lon': -96.9538, 'timezone': ...",17.33375,32.06
2,Bayeux,49.2764624,-0.7024738,"{'lat': 49.2765, 'lon': -0.7025, 'timezone': '...",16.96625,14.22
3,Le Havre,49.4938975,0.1079732,"{'lat': 49.4939, 'lon': 0.108, 'timezone': 'Eu...",16.32625,18.44
4,Rouen,49.4404591,1.0939658,"{'lat': 49.4405, 'lon': 1.094, 'timezone': 'Eu...",17.92250,31.45


In [13]:
# Ascending=False car nous voulons afficher des températures élevées et ascending=True car nous voulons de faibles précipitations pour identifier les meilleurs villes
nice_weather_cities = coordinates_df.sort_values(by=['AverageTemp', 'TotalPrecipitation'], ascending=[False, True])

In [14]:
# Transformation des données de temps en csv
nice_weather_cities.to_csv('nice_weather_cities.csv', index=False)

# Obtention des données par Scrapping.

Nous utilisons un script Python pour extraire des données d'hôtels depuis Booking.com à l'aide de la bibliothèque Scrapy. Le script initialise une Spider qui navigue sur les pages de résultats de recherche des villes françaises de notre liste. 

Pour chaque hôtel listé, le script extrait le nom, l'URL, la note, et la description, puis utilise une fonction auxiliaire pour récupérer les coordonnées géographiques de chaque hôtel via l'API Nominatim d'OpenStreetMap. Ces données sont ensuite compilées en un fichier JSON hotels.json. 

In [16]:
#!python scraping_booking.py

## Chargement et fusion des données

Dans cette etape, nous chargeons les données des hôtels depuis un fichier JSON dans un DataFrame Pandas, renommons certaines colonnes pour la clarté, et fusionnons ce DataFrame avec un autre contenant les coordonnées des villes. 

Le DataFrame final, qui combine les informations des hôtels et des villes, sera ensuite enregistré dans un fichier CSV et stocké dans un S3 pour des analyses futures.

In [17]:
# Lecture du fichier Json récupéré lors du scraping.
booking_df = pd.read_json('hotels.json')
booking_df.head()

,City,Hotel Name,Hotel URL,Score,Description,Latitude,Longitude
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396


In [18]:
# Rennomage des columns pour plus de clarté et éviter des confusion lors du merge des df.
booking_df.rename(columns={'Latitude': 'lat_hotels', 'Longitude': 'lon_hotels'}, inplace=True)
booking_df.head(1)

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.6147,-1.509617


In [19]:
# Fusion des df répertoriant les df et des villes en un seul df.
df_complete = pd.merge(booking_df, coordinates_df, on='City')
df_complete.head(1)

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,Weather,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.6147,-1.509617,48.6359541,-1.511459954959514,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",18.28375,23.87


In [20]:
# Transformation du df final en .csv
df_complete.to_csv('df_complete.csv', index=False)

## Affichage des cartes des 5 meilleurs villes et des 20 meilleurs hotels.

Comme demandée dans l'énoncé, nous affichons des 5 villes ayants les meilleurs températures dans un 1ere carte et les 20 hotels ayant les meilleurs score pour ces villes dans une seconde carte. Pour des soucis de précision et d'intéractivité, nous utiliserons la librairie Folium. 

In [57]:
# Plot des 5 meilleurs destinations de Frances selon la température.
# Crée une figure pour contenir la carte avec un titre
villes_fig = Figure(width=800, height=400)
villes_fig.html.add_child(folium.Element("<h3 style='text-align:center;margin-top:20px;'>Top 5 des villes avec la meilleure météo in France</h3>"))

# Crée une carte centrée autour de la France
map_france = folium.Map(location=[46.2276, 2.2137], zoom_start=6, tiles='OpenStreetMap')
villes_fig.add_child(map_france)

# Ajoute des marqueurs pour les top 5 destinations
for _, row in top_5_destinations.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['City']}: Temp {row['AverageTemp']}°C, Precip {row['TotalPrecipitation']}mm",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_france)

# Affiche la carte avec le titre
villes_fig




In [65]:
# Remplace les NaN dans la colonne 'Score' par 0 et élimine les lignes avec des coordonnées NaN
df_complete['Score'].fillna(0, inplace=True)
df_complete.dropna(subset=['lat_hotels', 'lon_hotels'], inplace=True)

# Filtre les hôtels pour ne garder que ceux dans les top 5 villes
top_cities = top_5_destinations['City'].tolist()
filtered_hotels = df_complete[df_complete['City'].isin(top_cities)]

# Sélectionne les 20 meilleurs hôtels pour chaque ville en fonction du score
top_hotels = filtered_hotels.groupby('City').apply(lambda x: x.nlargest(20, 'Score')).reset_index(drop=True)

In [66]:
# Crée une nouvelle figure pour la carte des hôtels
hotel_fig = Figure(width=800, height=400)
hotel_fig.html.add_child(folium.Element("<h3 style='text-align:center;margin-top:20px;'>Top 20 des hotels pour chacune des villes</h3>"))

# Crée une carte centrée autour de la France
hotel_map = folium.Map(location=[46.2276, 2.2137], zoom_start=6, tiles='OpenStreetMap')
hotel_fig.add_child(hotel_map)

# Ajoute des marqueurs pour les hôtels sélectionnés
for _, hotel in top_hotels.iterrows():
    folium.Marker(
        location=[hotel['lat_hotels'], hotel['lon_hotels']],
        popup=f"{hotel['Hotel Name']}: Score {hotel['Score']}<br><a href='{hotel['Hotel URL']}' target='_blank'>Book Now</a>",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(hotel_map)

# Affiche la carte avec les hôtels
hotel_fig


## Stocker le dataset dans un data lake S3

Nous configurons une session AWS S3 en utilisant nos identifiants d'accès spécifiques pour interagir avec le service de stockage cloud puis nous chargeonsle df complet en format csv  dans un bucket S3. 

Ensuite nous  lisons le fichier directement depuis S3 pour confirmer que les données sont correctement récupérées. Cette approche assure que les données sont correctement stockées et accessibles dans notre datalake sur S3.

In [38]:
# Charge la session S3 et les ressources nécessaires
session = boto3.Session(
    aws_access_key_id= # Votre clé d'accès AWS , 
    aws_secret_access_key= # Votre clé secrète AWS  
)
s3 = session.resource('s3')

In [39]:
# Upload le fichier dans S3
s3.Bucket('kayak-project-datalake').upload_file('df_complete.csv', 'data-kayak-project/df_complete.csv')

In [40]:
# Téléchargement et de lecture d'un fichier CSV depuis un bucket S3
try:
    obj = s3.Object('kayak-project-datalake', 'data-kayak-project/df_complete.csv')
    df_s3 = pd.read_csv(BytesIO(obj.get()['Body'].read()))
    print("File downloaded successfully")
except Exception as e:
    print(e)

File downloaded successfully


In [41]:
# Sanity check du df
df_s3.head(2)

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,Weather,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",18.28375,23.87
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",18.28375,23.87


## Transformation du dataframe

Dans cette section, nous voulons enrichir le DataFrame df_s3 avec des caractéristiques météorologiques détaillées extraites de la colonne Weather pour offrir une analyse plus approfondie des conditions climatiques de chaque lieu. Celà permet une meilleur lisibilitée et de tirer pleinement du potentiel d'information que contient la colonne "weather"

In [42]:
df_s3[['City', 'Hotel Name', 'Hotel URL', 'Score', 'Description', 'lat_hotels','lon_hotels', 'Latitude', 'Longitude', 'AverageTemp','TotalPrecipitation']]

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.511460,18.28375,23.87
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.511460,18.28375,23.87
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.635954,-1.511460,18.28375,23.87
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.635954,-1.511460,18.28375,23.87
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.635954,-1.511460,18.28375,23.87
...,...,...,...,...,...,...,...,...,...,...,...
871,La Rochelle,Première Classe La Rochelle Centre - Les Minimes,https://www.booking.com/hotel/fr/premia-re-cla...,7.5,Première Classe La Rochelle Centre - Les Minim...,46.147028,-1.153085,46.159113,-1.152043,19.10875,13.32
872,La Rochelle,360 degrés sur la rochelle,https://www.booking.com/hotel/fr/360-degres-su...,9.1,Managed by a private host,46.166189,-1.146359,46.159113,-1.152043,19.10875,13.32
873,La Rochelle,La Belle Amarre-Bed and Breakfast-Maison d'Hôtes,https://www.booking.com/hotel/fr/la-belle-amar...,9.4,Managed by a private host,46.158662,-1.155056,46.159113,-1.152043,19.10875,13.32
874,La Rochelle,Hôtel La Fabrique,https://www.booking.com/hotel/fr/la-fabrique-l...,8.4,Hôtel La Fabrique offers accommodation in La R...,46.156155,-1.148583,46.159113,-1.152043,19.10875,13.32


In [43]:
# Identification des informations contenues dans la colonne weather pour la 1ere ligne. 
print(df_s3['Weather'].iloc[0])

{'lat': 48.636, 'lon': -1.5115, 'timezone': 'Europe/Paris', 'timezone_offset': 7200, 'current': {'dt': 1718210589, 'sunrise': 1718165002, 'sunset': 1718222924, 'temp': 16.28, 'feels_like': 15.61, 'pressure': 1019, 'humidity': 63, 'dew_point': 9.23, 'uvi': 1.35, 'clouds': 4, 'visibility': 10000, 'wind_speed': 1.79, 'wind_deg': 331, 'wind_gust': 4.92, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}, 'daily': [{'dt': 1718193600, 'sunrise': 1718165002, 'sunset': 1718222924, 'moonrise': 1718186340, 'moonset': 1718150400, 'moon_phase': 0.2, 'summary': 'Expect a day of partly cloudy with rain', 'temp': {'day': 15.29, 'min': 7.38, 'max': 16.28, 'night': 10.7, 'eve': 15.76, 'morn': 11.04}, 'feels_like': {'day': 14.47, 'night': 9.78, 'eve': 15.06, 'morn': 10.29}, 'pressure': 1022, 'humidity': 61, 'dew_point': 7.73, 'wind_speed': 6.3, 'wind_deg': 319, 'wind_gust': 6.56, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clo

In [44]:
# Fonction pour extraire les données de la colonne weather
def extract_combined_weather_features(row):
    try:
        # Tente de convertir la chaîne météo en JSON, sinon convertit en dictionnaire
        if isinstance(row['Weather'], str) and row['Weather'].strip().startswith('{'):
            try:
                weather_data = json.loads(row['Weather'])
            except json.JSONDecodeError:
                weather_data = ast.literal_eval(row['Weather'].replace("'", '"'))
        else:
            weather_data = ast.literal_eval(row['Weather'])
        
        daily_data = weather_data['daily'] # Accède aux données journalières
        
       # Calculs pour les  mesures météorologiques à extraire
        max_temp = max(day['temp']['max'] for day in daily_data)
        min_temp = min(day['temp']['min'] for day in daily_data)
        avg_temp = sum(day['temp']['day'] for day in daily_data) / len(daily_data)
        avg_humidity = sum(day['humidity'] for day in daily_data) / len(daily_data)
        weather_descriptions = ', '.join(set(day['weather'][0]['description'] for day in daily_data))
        avg_wind_speed = sum(day['wind_speed'] for day in daily_data) / len(daily_data)
        max_uv = max(day['uvi'] for day in daily_data)
        total_precip = sum(day.get('rain', 0) for day in daily_data)
        avg_pressure = sum(day['pressure'] for day in daily_data) / len(daily_data)
        
        # Retourne toutes les mesures calculées sous forme de série
        return pd.Series([max_temp, min_temp, avg_temp, avg_humidity, weather_descriptions, avg_wind_speed, max_uv, total_precip, avg_pressure])
    except Exception as e:
        print(f"Error processing row: {e}")
        return pd.Series([None] * 9) # Retourne None pour chaque attribut en cas d'erreur

# Application de la fonction sur df_s3 pour enrichir les données
df_s3[['MaxTemp', 'MinTemp', 'AvgTemp', 'AvgHumidity', 'WeatherDesc', 'AvgWindSpeed', 'MaxUV', 'TotalPrecip', 'AvgPressure']] = df_s3.apply(extract_combined_weather_features, axis=1)



In [45]:
# Affiche les premières lignes pour vérifier les nouvelles données
df_s3.head()

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,Weather,...,TotalPrecipitation,MaxTemp,MinTemp,AvgTemp,AvgHumidity,WeatherDesc,AvgWindSpeed,MaxUV,TotalPrecip,AvgPressure
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125


Ces mesures météorologiques sont utiles pour optimiser les recommandations de destinations de voyage sur Kayak. Elles permettent d'évaluer le confort climatique (températures extrêmes et moyennes), le ressenti (humidité, pression), et les conditions météo potentielles (précipitations, vitesse du vent, UV), fournissant ainsi une base solide pour des suggestions personnalisées et pertinentes aux utilisateurs planifiant leurs voyages.

In [46]:
# Qualification en df_final_s3
df_final_s3= df_s3

## Stocker les données transformées dans une base de données RDS 

Nous établissons une connexion à une base de données PostgreSQL et y insérons df_final_s3 dans une table portant le même nom. 

Ensuite, nous exécutons une requête SQL pour récupérer et afficher les cinq premières lignes de la table afin de vérifier que les données ont été correctement transférées pour des analyses ultérieures.

In [47]:
# Connexion à la base de données PostgreSQL et insertion du DataFrame 'df_final_s3' dans la table 'df_final_s3'.
username = 'postgres'
password = 'omaromar'
host = 'kayak-project-db.c34qui6ye7rw.eu-north-1.rds.amazonaws.com'
port = '5432'
database = 'postgres'

# Création de l'engine de la database
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

# Transpose les DataFrames dans la database
table_name = 'df_final_s3'
df_final_s3.to_sql(table_name, engine, if_exists='replace', index=False)

876

In [48]:
# Test des datas avec une requête
query = text("SELECT * FROM df_final_s3 LIMIT 5")
pd.read_sql(query, engine)

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,Weather,...,TotalPrecipitation,MaxTemp,MinTemp,AvgTemp,AvgHumidity,WeatherDesc,AvgWindSpeed,MaxUV,TotalPrecip,AvgPressure
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.635954,-1.51146,"{'lat': 48.636, 'lon': -1.5115, 'timezone': 'E...",...,23.87,21.93,7.33,18.28375,69.375,"moderate rain, overcast clouds, light rain, sc...",7.02,6.68,23.87,1014.125
